# Impact of summation on update values
Ref: https://www.biorxiv.org/content/10.1101/2023.06.23.546237v1?med=mas.

In [25]:
using LinearAlgebra
using Polynomials

# example matrix
W = [0.0 0.9 0.0;
     0.9 0.0 0.1; 
     0.0 0.1 0.0]

# exmaple distance matrix
D = [0 3 4;
     3 0 5; 
     4 5 0.]

# example omega
ω = 0.9

# lets compute the derivative / gradient with respect to this 
demo_edge = [CartesianIndex(1, 2)];

In [19]:
function tangent_approx(
    f::Function, 
    W::Matrix{Float64},
    D::Matrix{Float64}, 
    ω::Float64,
    edges::Vector{CartesianIndex{2}}, 
    resolution = 0.01, steps = 5)::Vector{Float64}
    """
    Compute the derivative of any function f with respec to the vector of edges provided
    Will use n = steps number of nudges of current w_{edge idx} by (1+resolution)
    """
    results = zeros(length(edges))
    rep_vec = collect(range(-steps*resolution, steps*resolution, step=resolution))

    for (i_edge, edge_idx) in enumerate(edges)     
        # Points for evaluation
        edge_val = W[edge_idx]
        sign_edge = sign(edge_val) == 0 ? 1 : sign(edge_val)
        x = [edge_val + sign_edge * (max(abs(edge_val), 1e-3) * i) for i in rep_vec]
        
        # For each nudge save difference in communicability 
        y = zeros(length(x))
        for (i_rep, rep) in enumerate(x)
            W_copy = copy(W)
            W_copy[edge_idx] = rep 
            y[i_rep] = f(W_copy, D, ω, edge_idx)
        end

        results[i_edge] = fit(x, y, 1)[1]
    end

    return results
end;

## Version 1
$$
f(W) = \sum \sum ((c_{i,j} * d_{i,j})^{\omega})
$$
where
$$
C = e^{s^{-1/2} W s^{-1/2}}
$$

In [26]:
function obj_1(W, D, ω, _)
    node_strengths = dropdims(sum(W, dims=2), dims=2)
    node_strengths[node_strengths.==0] .= 1e-5
    S = sqrt(inv(Diagonal(node_strengths)))
    return sum((exp(S * W * S) .* D) .^ ω)
end;

tangent_approx(obj_1, W, D, ω, demo_edge, 0.01)

1-element Vector{Float64}:
 -0.0014622744308011133

## Version 2
$$
f(W) = ((\sum \sum c_{i,j}) * d_{i,j})^{\omega}
$$
with
$$
C = e^{s^{-1/2} W s^{-1/2}}
$$

Note, that this is like having one cost function for every edge individually, which are detached from each other, i.e.:
$$
f_{i,j}(W) = ((\sum \sum c_{i,j}) * d_{i,j})^{\omega}
$$

In [27]:
function obj_2(W, D, ω, edge)
    node_strengths = dropdims(sum(W, dims=2), dims=2)
    node_strengths[node_strengths.==0] .= 1e-5
    S = sqrt(inv(Diagonal(node_strengths)))
    return sum((exp(S * W * S)) .* D[edge]) .^ ω
end;

tangent_approx(obj_2, W, D, ω, demo_edge, 0.01)

1-element Vector{Float64}:
 -0.0012537066847978803